# Gather Twitter Data

## Import Dependencies

In [3]:
from config import (consumer_key, consumer_secret, access_token, access_token_secret)
from target_users import (target_users)

In [4]:
import  matplotlib.pyplot  as plt
import pandas as pd
import numpy as np
import tweepy
import json
import datetime
import time
import pprint as pprint
import requests
import json

%matplotlib inline

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

## Create Twitter Authorization

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

## Gather Twitter data through Twitter API and Tweepy

### Create Module to gather Tweets

In [36]:
def twttr_data ():
    #Create lists to hold tweets
    new_tweets = []
    public_tweets = []
    tweet_ids = []
    timestamp = []
    
    new_tweets = api.search('BTCUSD', count=100, result_type="recent")
    for tweet in new_tweets['statuses']:
        tweet_ids.append(tweet['id'])
        timestamp.append(tweet['created_at'])
        public_tweets.append(tweet['text'])
    
        oldest = tweet_ids[-1]
    
    for x in range(10):
        new_tweets = api.search('BTCUSD', count=100, result_type="recent", max_id = (oldest-1))
        #print(json.dumps(new_tweets['statuses'], indent = 4, sort_keys = True))
        for tweet in new_tweets['statuses']:
            tweet_id = tweet['id']
            #print(tweet_id)
            #if tweet_id not in tweet_ids:
            tweet_ids.append(tweet['id'])
            timestamp.append(tweet['user']['created_at'])
            public_tweets.append(tweet['text'])
    
        oldest = tweet_ids[-1]
    
    twitter_data = pd.DataFrame({'id':tweet_ids, 'timestamp':timestamp, 'tweets':public_tweets})
    twitter_data = twitter_data.sort_values('id', ascending = False)
    return(twitter_data)

### Create Sentiment Analysis using Vader Sentiment

In [21]:
def sentiment(twitter_data):
    #create list to hold sentiments
    compound_list = []
    sentiments = []
    
    count = 0
    
    for tweet in twitter_data['tweets']:
        results = analyzer.polarity_scores(tweet)
        compound = results["compound"]
        
        time = twitter_data['timestamp'][count]
        new_time = convert_time(time)
        
        sentiments.append({"Date": new_time, 
                           "Compound": compound,
                           "Text": tweet,
                           })
        count = count + 1
            

            
    #Create a dataframe from dict sentiments
    df = pd.DataFrame.from_dict(sentiments, dtype = str)
    df = df.sort_values(by = 'Date', ascending = True).reset_index(drop = True)
    return(df)

### Create Module to convert time stamp

In [22]:
def convert_time (time):
    new_time = datetime.datetime.strptime(time, "%a %b %d %H:%M:%S %z %Y").timestamp()
    new_time = datetime.datetime.fromtimestamp(new_time).strftime('%Y-%m-%d %H:%M:%S')
    return(new_time)

In [43]:
twitter_data = twttr_data()
Sentiment_pd = sentiment(twitter_data)

#Save dataframe into csv
sentiment_pd.to_csv('../Data_Sources/twitter_sentiments.csv')
twitter_data.to_csv('../Data_Sources/twitter_data.csv')

In [45]:
Sentiment_pd

,Compound,Date,Text
0,-0.4703,2006-12-11 12:38:07,"Closer up we have a breakout target of 6840, s..."
1,0.0,2007-07-31 00:12:41,RT @Pivotal_Pivots: Is the IHS on Bitcoin $BTC...
2,-0.5848,2007-08-11 03:11:33,RT @ClassicEthereum: 🔥 Drop #Altcoins : Buy $E...
3,0.1027,2007-11-10 11:30:35,Binance Exchange Backs First-Ever Decentralize...
4,0.6249,2008-03-08 09:58:18,Inverse head and shoulders pattern clearly in ...
5,0.5849,2008-03-13 10:18:53,@DJ_Erock23 @RonPaul Expressed in terms of dol...
6,0.0,2008-04-21 12:14:35,Have I not told you to go long on $btcusd fill...
7,0.3612,2008-05-17 14:19:23,"As previously mentioned, #BTCUSD was looking l..."
8,0.0,2008-06-11 04:36:37,RT @crypto_rand: Weekly $BTCUSD - $BTC chart:\...
9,0.0,2008-07-14 13:33:23,"@dKryptLive Heres my count for $BTC , i see a ..."
